https://colab.research.google.com/github/parrotProj/proj1/blob/master/modifed_res.ipynb

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
!git clone "https://github.com/parrotProj/proj1"

fatal: destination path 'proj1' already exists and is not an empty directory.


In [3]:
ls

proj1/  sample_data/


In [4]:
cd proj1/

/content/proj1


In [0]:
import Request
X_train, Y_train = Request.get_data.train()
X_test,X_id=Request.get_data.test()
np.random.seed(123577)

In [0]:
from skimage.transform import resize

for i in range(len(X_train)):
    if X_train[i].shape[0] != 150:
        X_train[i] = resize(X_train[i],(150,150,3))

for i in range(len(X_test)):
    if X_test[i].shape[0] != 150:
        X_test[i] = resize(X_test[i],(150,150,3))

In [0]:
def randomCrop(img, width, height):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
      
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]

    return img

In [0]:
import random
XX_train=[]
for i in range(len(X_train)):
    XX_train.append(randomCrop(X_train[i],150,150))

In [9]:
XX_train = np.array(XX_train)
XX_train.shape

(14034, 150, 150, 3)

In [0]:
XX_test=[]
for i in range(len(X_test)):
    XX_test.append(randomCrop(X_test[i],150,150))

In [11]:
XX_test=np.array(XX_test)
XX_test.shape

(3000, 150, 150, 3)

In [12]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

(14034,) (14034, 6)


In [13]:
from keras import Input
from keras.models import Model
from keras.layers import BatchNormalization, Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D, Dense, Activation, Dropout
from keras.layers import Add, ZeroPadding2D, Flatten

def input_block(x):
    
    x = ZeroPadding2D(padding=(2,2))(x)
    x = Convolution2D(64, (3,3), strides = (1,1), kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(padding=(1,1))(x)
    x = Convolution2D(64, (3,3), strides = (1,1), kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x


def residual_block(x,channels,d_increase=False):
    shortcut = x
  

    x = Convolution2D(int(channels/4.0), (1,1), padding = "valid", kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(int(channels/4.0), (3,3), padding = "same", kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(channels, (1,1), padding = "valid", kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    
    shortcut = Convolution2D(channels, (1,1), strides=(1,1), padding = "valid", kernel_initializer = 'he_normal')(shortcut)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x,shortcut])
    x = Activation('relu')(x)

    return x


def block_group(x, channels, num_blocks,d_increase=False):
    for i in range(num_blocks):
        if i==0 :
            x = residual_block(x,channels,d_increase)
        else:
            x = residual_block(x,channels,)
    
    return x

Using TensorFlow backend.


In [0]:
def result(x, channels=64):
    input_data=x            #150 * 150 *3
    x = input_block(x)
    x = MaxPooling2D((3,3),2)(x)

    x = block_group(x, 128, 5, d_increase=False)
    x = MaxPooling2D((2,2),2)(x)

    x = block_group(x, 256, 7, d_increase=True)
    x = MaxPooling2D((2,2),2)(x)

    x = block_group(x, 512, 4, d_increase=True)
    
    x = GlobalAveragePooling2D()(x)
    output_data = Dense(6,activation='softmax')(x)
    
    model = Model(input_data,output_data)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    return model

In [15]:
inputs = Input(shape=(150,150,3),dtype='float32')
model = result(inputs)

In [16]:
#hist=model.fit(x=XX_train,y=Y_train,validation_split=0.2,batch_size=56,epochs=100)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 154, 154, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 152, 152, 64) 1792        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 152, 152, 64) 256         conv2d_1[0][0]                   
____________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range=0.05, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True, zoom_range=0.2)
from sklearn.model_selection import train_test_split

X_train, X_Val, Y_train, Y_Val = train_test_split(XX_train,Y_train, test_size=0.1)

In [0]:
train_generator = train_datagen.flow(X_train,Y_train,batch_size=64)
hist=model.fit_generator(train_generator,steps_per_epoch=128,validation_data=(X_Val,Y_Val),
               validation_steps=32,epochs=30)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
128/128 [==============================] - 109s 849ms/step - loss: 1.2129 - acc: 0.5170 - val_loss: 5.1756 - val_acc: 0.2044
Epoch 2/30
128/128 [==============================] - 79s 616ms/step - loss: 0.8873 - acc: 0.6467 - val_loss: 4.3050 - val_acc: 0.4152
Epoch 3/30
 29/128 [=====>........................] - ETA: 57s - loss: 0.7977 - acc: 0.6994

In [0]:
fig,loss_ax=plt.subplots(figsize=(12,5))
acc_ax=loss_ax.twinx()

loss_ax.plot(hist.history['loss'],'indianred',marker='*',label='train loss')
loss_ax.plot(hist.history['val_loss'],'lightcoral',marker='*',label='val loss')

acc_ax.plot(hist.history['acc'],'forestgreen',marker='*',label='train accuary')
acc_ax.plot(hist.history['val_acc'],'olivedrab',marker='*',label='val accuary')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [0]:
pred=model.predict(X_Val,batch_size=32)
result = np.argmax(pred,axis=1).reshape(-1,1)

In [0]:
result = model.predict(XX_test,batch_size=32)
result = np.argmax(result,axis=1).reshape(-1,1)

X_id_pd = pd.DataFrame(X_id)
X_id_pd = X_id_pd[0].apply(lambda x:x.split('.')[0])
output = pd.DataFrame(X_id_pd)

In [0]:
output['pred_label'] = result
output.columns = ['id','pred_label']
output.set_index('id')

In [0]:
output.to_csv('prediction.csv')

from google.colab import files
files.download("prediction.csv")

In [0]:
## Convolution : 컨볼루션 레이어, 필터로 특징을 뽑아주는 레이어.

## Convolution2D(컨볼루션 필터의 수, 컨볼루션 커널의 (행,열), padding, input_shape)
##               필터는 가중치를 의미하며, 하나의 필터가 입력 이미지를 순회하면서 적용된 결과값을 모으면 출력이미지가 생성됨
##               순회할 때 적용되는 가중치는 모두 동일함 (파라미터 공유) => 학습해야할 가중치를 줄임, 이미지 인식에 적합

## padding : 경계 처리 방법을 정의. 'valid' : 유효한 영역만 출력. 출력 이미지 사이즈는 입력 사이즈보다 작음
##                                 'same' : 출력 이미지 사이즈가 입력 이미지 사이즈와 동일.

## input_shape : 샘플 수를 제외한 입력 형태를 정의. 첫 레이어에서만 정의하면 된다.
##               (행,열,채널 수)로 정의함. 흑백인 경우 채널 = 1,컬러인 경우 채널 = 3




## MaxPooling : 맥스풀링 레이어. 사소한 변화를 무시해준다. (지엽적인 부분이 영향을 미치지 않도록 한다.)
##              컨볼루션 레이어의 출력 이미지에서 주요값만 뽑아 크기가 작은 출력 이미지를 만든다
## MaxPooling2D(pool_size)
## pool_size : 수직, 수평 수평 축소 비율을 지정 ((2,2)면 출력 영상 크기는 입력 영상 크기의 반으로 줄어든다)


## 컨볼루션 레이어나 맥스풀링 레이어를 반복적으로 수행하면 주요 특징만 추출된다
## 추출된 주요 특징은 전결합층에 전달되어 학습된다.
## 컨볼루션과 맥스풀링은 2차원 데이터를 다루지만 전결합층에 전달하기 위해서는 1차원 자료로 바꿔줘야한다.
## Flatten : 1차원 자료로 바꿔주는 레이어

In [0]:
output['count']=1
output.groupby('pred_label').sum()

In [0]:
output['count']=1
output.groupby('pred_label').sum()